In [1]:
import pandas as pd
import glob
import os
from json import loads, dumps
import json



#combining all the daily gamelogs csv files pulled
path = 'filepath' 
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

dfall = pd.concat(li, axis=0, ignore_index=True)

In [2]:
dfall['GAME_DATE']=dfall['GAME_DATE'].apply(pd.to_datetime)

In [3]:
dfallsort=dfall.sort_values(by=['GAME_DATE','GAME_ID']).reset_index(drop=True)

In [4]:
dfgamenum = dfallsort.copy()

In [5]:
dfgamenum['GAME_NO']=dfgamenum.groupby(['TEAM_ID'])['GAME_DATE'].cumcount() + 1

In [6]:
dfgamenum=dfgamenum.copy()
dfdata=dfgamenum

In [7]:
offptsvar = dfdata.groupby('TEAM_ID')['PTS'].expanding().sum().reset_index(0,drop=True)
offposvar = dfdata.groupby('TEAM_ID')['POSS'].expanding().sum().reset_index(0,drop=True)

defptsvar = dfdata.groupby('TEAM_ID')['OPP_PTS'].expanding().sum().reset_index(0,drop=True)
defposvar = dfdata.groupby('TEAM_ID')['OPP_POSS'].expanding().sum().reset_index(0,drop=True)

In [8]:
dfdata['STD_OFF_RATING']=(offptsvar/offposvar)*100
dfdata['STD_OFF_RATING']=dfdata['STD_OFF_RATING'].round(1)

dfdata['STD_DEF_RATING']=(defptsvar/defposvar)*100
dfdata['STD_DEF_RATING']=dfdata['STD_DEF_RATING'].round(1)

In [9]:
#Creating JSON to reference for last N Games
dfarray = dfdata[["TEAM_CITY","GAME_NO","PTS","POSS","OPP_PTS","OPP_POSS"]]


dflastn=dfarray.groupby(['TEAM_CITY'])[['TEAM_CITY',"GAME_NO","PTS","POSS","OPP_PTS","OPP_POSS"]].apply(lambda x: x.to_dict('records')).reset_index(name="TEAMS")
dfarraygrouplastn=dflastn[["TEAMS"]]


In [10]:
dfarraygroupjsonlastn= dfarraygrouplastn.to_json(orient="records")


In [11]:
parsedjsonlastn = json.loads(dfarraygroupjsonlastn)


In [12]:
dflg=dfdata[['GAME_NO','PTS','POSS']]

In [13]:
dflg=dflg.groupby("GAME_NO").sum().reset_index()

In [14]:
dflg['PTS']=dflg['PTS'].cumsum()
dflg['POSS']=dflg['POSS'].cumsum()

In [15]:
dflg=dflg.copy()

In [16]:
dflg['RTG']=(dflg['PTS']/dflg['POSS'])*100


In [17]:
#Rankign calcuation

dfrankingdata=dfgamenum
dfrankingdata['STD_OFF_RATING']=(offptsvar/offposvar)*100
dfrankingdata['STD_OFF_RATING']=dfrankingdata['STD_OFF_RATING']

dfrankingdata['STD_DEF_RATING']=(defptsvar/defposvar)*100
dfrankingdata['STD_DEF_RATING']=dfrankingdata['STD_DEF_RATING']

dflastgame = dfrankingdata.sort_values('GAME_NO').groupby('TEAM_CITY').tail(1).reset_index(drop="True")

dfrank = dflastgame[["TEAM_CITY","STD_OFF_RATING","STD_DEF_RATING"]]

dfrank=dfrank.copy()
dfrank['DEF_RANKING'] = dfrank['STD_DEF_RATING'].rank().astype(int)
dfrank['OFF_RANKING'] = dfrank['STD_OFF_RATING'].rank(ascending=False).astype(int)

In [18]:
#Creating Json for Team Rank for Off and Def Rating
dfrankcols = dfrank[['TEAM_CITY','OFF_RANKING','DEF_RANKING']]
dfrankcols=dfrankcols.copy()
dfrankcols.sort_values(by='TEAM_CITY',inplace=True)

In [19]:
dfrankcols.set_index('TEAM_CITY', inplace=True)
dfrankjson=dfrankcols.to_json(orient="index")
dfrankparsed = json.loads(dfrankjson)

In [20]:
dfdatajsoncols = dfdata[['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
                         'POSS', 'GAME_DATE', 'PTS', 'OPP_TEAM_ID', 
                         'OPP_PTS', 'OPP_POSS', 'OPP_TEAM_NAME', 'OPP_TEAM_ABBERVIATION',
                         'OPP_TEAM_CITY', 'GAME_NO', 'STD_OFF_RATING', 'STD_DEF_RATING']]

In [21]:
dfdatajsoncols=dfdatajsoncols.copy()

dfdatajsoncols['GAME_DATE'] = dfdatajsoncols['GAME_DATE'].astype(str)

In [22]:
cols=['GAME_ID','POSS', 'GAME_DATE', 'PTS', 'OPP_TEAM_ID', 
                         'OPP_PTS', 'OPP_POSS', 'OPP_TEAM_NAME', 'OPP_TEAM_ABBERVIATION',
                         'OPP_TEAM_CITY', 'GAME_NO', 'STD_OFF_RATING', 'STD_DEF_RATING']

In [23]:
dfb=dfdatajsoncols.groupby(['TEAM_NAME','TEAM_ID','TEAM_CITY','TEAM_ABBREVIATION'])[cols].apply(lambda x: x.to_dict('records')).reset_index(name='GAME_INFO')

In [24]:
dflg2= dflg[['GAME_NO','RTG']]

In [25]:
jsonlgavgdata=dflg2.to_json(orient="records")

In [26]:
dfb.set_index('TEAM_CITY', inplace=True)

In [27]:
jsondata=dfb.to_json(orient="index")


In [28]:
parsed = json.loads(jsondata)


In [29]:


parsedlgavg = json.loads(jsonlgavgdata)


In [30]:
#combine jsons into single JSON to reference
combined = [parsedlgavg, parsed,parsedjsonlastn ,dfrankparsed]

In [31]:

with open('filepath', 'w', encoding='utf-8') as f:
    json.dump(combined, f, ensure_ascii=False)